In [31]:
import sys
import mido as md 
from mido import MidiFile
from collections import defaultdict
import numpy as np
from sklearn.preprocessing import normalize

song = md.MidiFile('chpn_op10_e05.mid')
trackNotes = defaultdict(list)
pMatrices = defaultdict(list)
for i, track in enumerate(song.tracks):
    pMatrix = np.zeros((128, 128))
    prevNote = None
    for msg in track:
        if msg.type == 'note_on' and msg.velocity:
            if prevNote:
                pMatrix[prevNote][msg.note] += 1
            prevNote = msg.note
    if pMatrix.any():
        pMatrices[i] = normalize(pMatrix, axis =1, norm = 'l1')
print(pMatrices[1])

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [32]:
songLength = 500
starting_note = 66

In [43]:
from mido import Message, MidiFile, MidiTrack

delta = 80
current_note = starting_note
current_time = 0

rh_trans = pMatrices[1]

# print(rh_trans[current_note])

outfile = MidiFile()

track = MidiTrack()
outfile.tracks.append(track)

# track.append(Message('instrument_name', 'piano'))
track.append(Message('program_change', program=12))

for i in range(songLength):
    track.append(Message('note_on', note = current_note, velocity = 60, time = delta))
    possibleNexts = {i:rh_trans[current_note][i] for i in range(128) if rh_trans[current_note][i] > 0}
    notes = list(possibleNexts.keys())
    probabilities = [possibleNexts[n] for n in notes]
    current_note = np.random.choice(notes, 1, p=probabilities)[0]

outfile.save('testChopin.mid')
